In [56]:
import pygame
import random

# 게임 설정
pygame.init()

# 화면 크기
width, height = 300, 600
block_size = 30
cols, rows = width // block_size, height // block_size

images = {
    "ice": pygame.transform.scale(pygame.image.load("img/ice_block.jpg"), (block_size, block_size)),
    "fire": pygame.transform.scale(pygame.image.load("img/fire_block.jpg"), (block_size, block_size)),
    "glass": pygame.transform.scale(pygame.image.load("img/glass_block.jpg"), (block_size, block_size)),
    "stone": pygame.transform.scale(pygame.image.load("img/stone_block.jpg"), (block_size, block_size)),
    "tree": pygame.transform.scale(pygame.image.load("img/tree_block.jpg"), (block_size, block_size))
}

# 테트리스 블록의 모양과 색상을 정의합니다.
# 이전의 colors 리스트 대신 images 딕셔너리를 사용합니다.
tetrominoes = [
    {"shape": [[1, 1, 1, 1]], "color": "ice"},  # I
    {"shape": [[1, 1], [1, 1]], "color": "fire"},  # O
    {"shape": [[0, 1, 0], [1, 1, 1]], "color": "glass"},  # T
    {"shape": [[1, 1, 0], [0, 1, 1]], "color": "stone"},  # S
    {"shape": [[0, 1, 1], [1, 1, 0]], "color": "tree"},  # Z
    {"shape": [[1, 0, 0], [1, 1, 1]], "color": "ice"},  # J
    {"shape": [[0, 0, 1], [1, 1, 1]], "color": "fire"}   # L
]



# 블록 클래스 정의
class Tetromino:
    def __init__(self, shape_data):
        self.shape = shape_data["shape"]  # 딕셔너리에서 모양 데이터 추출
        self.color = shape_data["color"] 
        self.image = get_img(shape_data)  # 해당 모양의 색상 가져오기
        self.x = cols // 2 - len(self.shape[0]) // 2
        self.y = 0

    def rotate(self):
        self.shape = list(zip(*self.shape[::-1]))
        self.image = pygame.transform.rotate(self.image, -90)  # 이미지 회전


def get_img(tetromino):
    return images[tetromino.color]

def create_grid(locked_positions={}):
    grid = [[images["ice"] for _ in range(cols)] for _ in range(rows)]  # 이미지 대신 색상 사용
    for y in range(rows):
        for x in range(cols):
            if (x, y) in locked_positions:
                c = locked_positions[(x, y)]
                grid[y][x] = c
    return grid

# colors 리스트 대신 이미지를 사용하여 격자를 그리는 함수 수정
def draw_grid(surface, grid):
    for y in range(rows):
        for x in range(cols):
            surface.blit(grid[y][x], (x * block_size, y * block_size))  # 이미지를 격자에 그립니다.
    pygame.draw.rect(surface, (255, 255, 255), (0, 0, width, height), 5)

def get_shape():
    random_piece = random.choice(tetrominoes)
    return random_piece


# 테트로미노의 회전 상태에 따라 각 블록의 위치를 계산하고, 게임 보드에서의 위치를 반환합니다.
# 여기가 촘촘한 격자를 갖도록 해야함.
def valid_space(shape, grid):
    accepted_positions = [[(x, y) for x in range(cols) if grid[y][x] == images["ice"]] for y in range(rows)]
    accepted_positions = [x for sub in accepted_positions for x in sub]
    formatted = convert_shape_format(shape)
    for pos in formatted:
        if pos not in accepted_positions:
            if pos[1] > -1:
                return False
    return True


# 방향전환
def convert_shape_format(shape):
    positions = []
    for y, line in enumerate(shape["shape"]):
        for x, cell in enumerate(line):
            if cell == 1:
                positions.append((x, y))
    return positions

# 게임 오버 조건 확인 > 고칠 필요 있음, 시간제한 1분 추가
def check_lost(positions):
    for pos in positions:
        x, y = pos
        if y < 1:
            return True
    return False

def clear_rows(grid, locked):
    inc = 0
    for y in range(rows-1, -1, -1):
        row = grid[y]
        if images["ice"] not in row:
            inc += 1
            ind = y
            for x in range(cols):
                try:
                    del locked[(x, y)]
                except:
                    continue
    if inc > 0:
        for key in sorted(list(locked), key=lambda k: k[1])[::-1]:
            x, y = key
            if y < ind:
                newKey = (x, y + inc)
                locked[newKey] = locked.pop(key)
    return inc


def draw_next_shape(shape, surface):
    font = pygame.font.SysFont('comicsans', 30)
    label = font.render('Next Shape', 1, (0, 0, 0))
    sx = width + 50
    sy = height / 2 - 100
    format = shape.shape
    for y, line in enumerate(format):
        for x, cell in enumerate(line):
            if cell == 1:
                surface.blit(get_img(shape), (sx + x * block_size, sy + y * block_size))  # 이미지를 격자에 그립니다.
    surface.blit(label, (sx + 10, sy - 30))

def draw_window(surface, grid, score=0):
    surface.fill((255, 255, 255))
    font = pygame.font.SysFont('comicsans', 60)
    label = font.render('Tetris', 1, (0, 0, 0))
    surface.blit(label, (width / 2 - (label.get_width() / 2), 30))
    font = pygame.font.SysFont('comicsans', 30)
    label = font.render('Score: ' + str(score), 1, (0, 0, 0))
    sx = width + 50
    sy = height / 2 - 100
    surface.blit(label, (sx + 20, sy + 160))
    draw_grid(surface, grid)


def main():
    locked_positions = {}  # 고정된 블록의 위치와 색상을 저장하는 딕셔너리
    grid = create_grid(locked_positions)  # 현재 게임 보드를 생성
    change_piece = False  # 블록을 교체해야 하는지 여부를 나타내는 변수
    run = True  # 게임 루프를 제어하는 변수
    current_piece = get_shape()  # 현재 떨어지고 있는 블록을 생성
    next_piece = get_shape()  # 다음에 떨어질 블록을 생성
    clock = pygame.time.Clock()  # 게임 속도 조절을 위한 시계 객체 생성
    fall_time = 0  # 블록이 떨어지는 시간을 추적
    level_time = 0  # 레벨 증가 시간을 추적
    score = 0  # 게임 점수를 추적

    while run:
        grid = create_grid(locked_positions)  # 고정된 블록을 포함하여 게임 보드를 업데이트
        fall_speed = 0.27  # 초기 블록 낙하 속도 설정
        fall_time += clock.get_rawtime()  # 지난 프레임의 시간을 더함
        level_time += clock.get_rawtime()  # 지난 프레임의 시간을 더함
        clock.tick()  # 매 프레임 사이의 시간을 조정

        # 레벨 시간에 따라 블록 낙하 속도 조절
        if level_time / 1000 > 5:  # 레벨 시간이 5초를 초과하면
            level_time = 0  # 레벨 시간 초기화
            if fall_speed > 0.12:  # 낙하 속도가 최소값보다 크면
                fall_speed -= 0.005  # 낙하 속도를 줄임

        # 블록이 떨어지는 로직
        if fall_time / 1000 >= fall_speed:  # 낙하 시간이 낙하 속도보다 크거나 같으면
            fall_time = 0  # 낙하 시간 초기화
            current_piece.y += 1  # 블록을 한 칸 아래로 이동
            if not (valid_space(current_piece, grid)) and current_piece.y > 0:  # 유효한 위치가 아니면
                current_piece.y -= 1  # 다시 한 칸 위로 이동
                change_piece = True  # 블록을 교체해야 함

            # 사용자 입력 이벤트 처리
        for event in pygame.event.get():
            if event.type == pygame.QUIT:  # 게임 창 닫기 이벤트
                run = False  # 게임 루프 종료
                pygame.display.quit()  # 게임 창 닫기
                quit()  # 프로그램 종료
            if event.type == pygame.KEYDOWN:  # 키보드 입력 이벤트
                if event.key == pygame.K_LEFT:  # 왼쪽 화살표 키
                    current_piece.x -= 1  # 블록을 왼쪽으로 이동
                    if not valid_space(current_piece, grid):  # 유효한 위치가 아니면
                        current_piece.x += 1  # 다시 오른쪽으로 이동
                if event.key == pygame.K_RIGHT:  # 오른쪽 화살표 키
                    current_piece.x += 1  # 블록을 오른쪽으로 이동
                    if not valid_space(current_piece, grid):  # 유효한 위치가 아니면
                        current_piece.x -= 1  # 다시 왼쪽으로 이동
                if event.key == pygame.K_DOWN:  # 아래쪽 화살표 키
                    current_piece.y += 1  # 블록을 아래로 빠르게 이동
                    if not valid_space(current_piece, grid):  # 유효한 위치가 아니면
                        current_piece.y -= 1  # 다시 위로 이동
                if event.key == pygame.K_UP:  # 위쪽 화살표 키
                    current_piece.rotate()  # 블록을 회전
                    if not valid_space(current_piece, grid):  # 유효한 위치가 아니면
                        current_piece.rotate()  # 회전을 취소 (총 4번 회전해서 원래 상태로)
                        current_piece.rotate()
                        current_piece.rotate()
                if event.key == pygame.K_SPACE:  # 스페이스 바
                    # 블록을 맨 아래로 이동
                    while valid_space(current_piece, grid):  
                        current_piece.y += 1  # 블록을 아래로 이동
                    # 한 칸 위로 이동 (유효한 위치를 벗어나므로)
                    current_piece.y -= 1
                    # 블록을 고정시키고 새로운 블록 생성
                    for pos in convert_shape_format(current_piece):
                        p = (pos[0], pos[1])
                        locked_positions[p] = get_img(current_piece)  # 이미지를 가져와서 고정
                    current_piece = next_piece  
                    next_piece = get_shape()  
                    score += clear_rows(grid, locked_positions) * 10


        shape_pos = convert_shape_format(current_piece)  # 현재 블록의 좌표 계산

        # 블록의 좌표를 그리드에 업데이트
        for i in range(len(shape_pos)):
            x, y = shape_pos[i]
            if y > -1:  # 그리드 범위 내에 있는 좌표만 업데이트
                grid[y][x] = get_img(current_piece)  # 이미지를 가져와서 업데이트

        # 블록 교체가 필요하면
        if change_piece:
            for pos in shape_pos:
                p = (pos[0], pos[1])
                locked_positions[p] = get_img(current_piece)  # 이미지를 가져와서 고정된 블록의 위치 업데이트
            current_piece = next_piece  # 현재 블록을 다음 블록으로 교체
            next_piece = get_shape()  # 새로운 다음 블록 생성
            change_piece = False  # 블록 교체 플래그 초기화
            score += clear_rows(grid, locked_positions) * 10  # 점수 업데이트


        draw_window(win, grid, score)  # 게임 화면 그리기
        draw_next_shape(next_piece, win)  # 다음 블록 표시
        pygame.display.update()  # 화면 업데이트

        # 게임 오버 조건 확인
        if check_lost(locked_positions):
            run = False  # 게임 루프 종료

    pygame.display.quit()  # 게임 창 닫기

def main_menu():
    run = True
    while run:
        win.fill((255, 255, 255))
        font = pygame.font.SysFont('comicsans', 60)
        label = font.render('Press Any Key To Play', 1, (0, 0, 0))
        win.blit(label, (width / 2 - label.get_width() / 2, height / 2 - label.get_height() / 2))
        pygame.display.update()
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                run = False
            if event.type == pygame.KEYDOWN:
                main()
    pygame.quit()

In [57]:
win = pygame.display.set_mode((width + 200, height))
pygame.display.set_caption('Impossible_Tetris')
main_menu()

AttributeError: 'dict' object has no attribute 'color'

In [58]:
pygame.display.quit()